# **HR Resources EDA**

![500,400](https://images.pexels.com/photos/1170412/pexels-photo-1170412.jpeg?cs=srgb&dl=pexels-cadeau-maestro-1170412.jpg&fm=jpg)

# Dependencies

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder as LE
import altair as alt
import torch
import torchvision.transforms as transforms
alt.data_transformers.disable_max_rows()#to make charts above 5000 rows

# Importing And Understanding the Dataset
lets take a look and understand the dataset

In [ ]:
df = pd.read_csv("../input/hr-analytics-and-job-prediction/HR_comma_sep.csv")
df

In [ ]:
display(df.info())
display(df.describe())

Let's look at this column by column!

satisfaction_level : This is most likely the Employee satisfaction level

last_evaluation : The score that the employee got for his evaluation

number_projects: Number of projects done by the employee

average_monthly_hours: Average number of hours done with respect to 30 days

time_spend_company: Assuming the number of years spent in the company

Work_accident : Weather or not a accident has occurred in the workplace

left:weather or not the employee has left the company 1 if left 0 if not

promotion_last_5years: weather or not the employee has had a promotion 1 if promoted 0 if not

Department: Department in the company

salary: Degree of salary High,Medium or Low

Looking at this Dataset it has been cleaned throughly so I won't bother with Cleaning

# Using Barplots and Histograms to understand data

In [ ]:
categorical_data = ['Work_accident','number_project','promotion_last_5years','Department','salary','time_spend_company']
numerical_data = ['satisfaction_level', 'last_evaluation','average_montly_hours']
target = "left"
def bar_chart_maker(df,xax,yax,colorax=""):
    chart = alt.Chart(df).mark_bar().encode(
        x = xax,
        y = yax,
        color = colorax
    )
    display(chart)

In [ ]:
df.keys()

In [ ]:
bar_chart_maker(df,'left',"count()","satisfaction_level")

Hmmm the number of employees who left are quite less than the number of employees who didn't this might be a problem while making the model.The employees who left also seem to have a lower satisfaction level

In [ ]:
bar_chart_maker(df,'number_project',"count()","left")

This looks like a typical Guassian Distribution all employees have done more than 1 project. Most employees have done 4 projects.

In [ ]:
bar_chart_maker(df,'time_spend_company',"count()","left")

A large number of employees have spent 3 years in the company. However the employee retention seems to decrease after 3 years. The most senior employee has spent 10 years in the company. This looks like a typical tech comapny , after gaining a bit of experience most employees tend to leave for better paying jobs.

In [ ]:
bar_chart_maker(df,'Work_accident',"count()","left")

nothing surprising here the number of accidents in the workplace seem quite small compared to the larger dataset.

In [ ]:
bar_chart_maker(df,"count()",'promotion_last_5years',"left")

out of 14999 employees less than 1000 have recieved promotion surprisingly however the number of employees who stayed after getting the promotion is low this needs to be checked further

In [ ]:
bar_chart_maker(df,"count()",'Department',"left")

In [ ]:
bar_chart_maker(df,"count()",'Department','promotion_last_5years')

Sales seem to have the highest number of promotions followed by management and marketing. Promotions in IT accounting and HR seems to be very low

In [ ]:
bar_chart_maker(df,"count()",'salary','left')

Most employees that leave seem to have low or medium salaries. Unsurprisingly most high payed employees don't leave 

In [ ]:
def histogram_maker(num_category,colorax,max_ = 1):
    plot = alt.Chart(df).mark_bar().encode(
        alt.Y(num_category, bin= {"binned": False, "step": 0.05 , "extent" : [0,max_]}),
        x='count()',
        color = colorax
    )
    display(plot)

In [ ]:
#satisfaction_level last_evaluation
histogram_maker("satisfaction_level","left")

most employees who leave seem to have a satisfaction level of 0.5 and below.

In [ ]:
histogram_maker("last_evaluation","left")

This graph is pretty arbitary don't know what to make of it.

# Transforming and Feature Selection

In [ ]:
X = df
y = df['left']

In [ ]:
cat_features = ['Department', 'salary']
features = ['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'Department', 'salary']

In [ ]:
X = df[features]
le = LE()
for x in cat_features:
    X[x] = le.fit_transform(X[x])

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2 , f_classif
select_features = SelectKBest( f_classif,k = 5).fit(X,y)
selected_features_df = pd.DataFrame({
    "Features" : features,
    "Scores" : select_features.scores_
})
selected_features_df

In [ ]:
bar_chart_maker(selected_features_df,"Features",'Scores','Scores')

Satisfaction_level seems to be the most important feature followed by work_accident and promotion last 5 years
i won't count time_spend_company since those who don't leave will spend a larger amount of time in the company. Surprisingly the salary seems to be the least important feature to determine weather the employee will leave or not.

I'll be doing Feature engineering after making a baseline model on . Comment what you think of this EDA